In [1]:
import numpy as np
import pandas as pd
import spacy
from nltk.corpus import stopwords
from string import punctuation

In [2]:
comments_df = pd.read_csv('./comments_with_score.csv',index_col=0)

In [3]:
comments_df.head()

,article_id,comments,is_reply,neg,neu,pos,compound
0,0.0,What's the point of studying so much ended up ...,0.0,0.000,0.872,0.128,0.7096
1,0.0,No matter what kind of streaming or subject ba...,0.0,0.156,0.760,0.084,-0.8555
2,0.0,Seems to be that the purpose of this system is...,1.0,0.000,0.844,0.156,0.6322
3,0.0,This feels like just another diversion from RE...,0.0,0.045,0.797,0.159,0.8981
4,0.0,Isn’t a “real” issue the boxing of kids into s...,1.0,0.000,0.690,0.310,0.6597


## Gensim LDA with lemmatization

In [4]:
#Get english stopwords from nltk
nltk_stops = stopwords.words('english')

In [5]:
#Instantiate tokenizer
nlp = spacy.load('en_core_web_lg')

In [6]:
#Add curly double quotes to punctuation list
punctuation = punctuation + '“' + '”'

In [7]:
def clean_text(article):
    '''
    This function takes in a string and returns the string with stopwords and punctuation removed, plus words are lemmatized unless they are pronouns.
    '''
    #Remove stopwords and punctuation
    removed_stopwords = " ".join(i for i in article.lower().split() if i not in nltk_stops)
    removed_punctuation = "".join(i for i in removed_stopwords if i not in punctuation)
    
    #Tokenize words
    doc = nlp(removed_punctuation)
    #Lemmatize words
    lemmatized = " ".join(token.lemma_ if token.pos_ != 'PRON' else token.text for token in doc )
    return lemmatized

In [8]:
#Try cleaning the first comment
clean_text(comments_df['comments'][0])

'what s point study much end work clean taxi driver university graduate rest position fill foreign talent work apply singapore citizenship go back china join people liberation army drop singapore citizenship'

In [9]:
#Add the cleaned comments to the DataFrame
comments_df['cleaned comments'] = comments_df['comments'].apply(clean_text)

In [10]:
import gensim
from gensim import corpora,models

In [11]:
#Convert each comment into a list of words
text_list = [text.split() for text in comments_df['cleaned comments']]

# Create the term dictionary of our corpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(text_list)
print(dictionary)

Dictionary(2135 unique tokens: ['apply', 'army', 'back', 'china', 'citizenship']...)


In [12]:
#Convert corpus into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]

In [13]:
#Create the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

#Run and train LDA model on the document term matrix for 2 topics
ldamodel = Lda(doc_term_matrix, num_topics=2, id2word = dictionary, passes = 100, random_state = 42)

In [14]:
from pprint import pprint

In [15]:
#Print topic number and most contributing words
pprint(ldamodel.print_topics(num_words=15))

[(0,
  '0.013*"student" + 0.011*"school" + 0.010*"stream" + 0.009*"good" + '
  '0.007*"go" + 0.006*"take" + 0.006*"education" + 0.006*"system" + '
  '0.006*"change" + 0.006*"one" + 0.005*"policy" + 0.004*"parent" + '
  '0.004*"minister" + 0.004*"child" + 0.004*"study"'),
 (1,
  '0.011*"singapore" + 0.009*"like" + 0.007*"even" + 0.006*"bad" + '
  '0.006*"many" + 0.006*"not" + 0.006*"student" + 0.006*"singaporean" + '
  '0.006*"people" + 0.006*"system" + 0.006*"stream" + 0.005*"chinese" + '
  '0.005*"year" + 0.005*"normal" + 0.005*"get"')]


In [16]:
#Run and train LDA model on the document term matrix for 3 topics
ldamodel3 = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes = 100, random_state = 42)

In [17]:
#Print topic number and most contributing words
pprint(ldamodel3.print_topics(num_words=15))

[(0,
  '0.013*"school" + 0.010*"student" + 0.010*"stream" + 0.007*"good" + '
  '0.007*"one" + 0.006*"go" + 0.006*"normal" + 0.005*"system" + 0.005*"class" '
  '+ 0.005*"uturn" + 0.005*"child" + 0.005*"like" + 0.005*"education" + '
  '0.004*"lose" + 0.004*"many"'),
 (1,
  '0.012*"singapore" + 0.010*"like" + 0.008*"student" + 0.007*"bad" + '
  '0.007*"system" + 0.007*"stream" + 0.007*"even" + 0.007*"year" + '
  '0.006*"chinese" + 0.006*"many" + 0.006*"singaporean" + 0.006*"people" + '
  '0.005*"education" + 0.005*"kid" + 0.005*"government"'),
 (2,
  '0.010*"good" + 0.009*"student" + 0.009*"take" + 0.008*"get" + '
  '0.007*"change" + 0.006*"stream" + 0.006*"policy" + 0.006*"school" + '
  '0.006*"education" + 0.006*"streaming" + 0.005*"people" + 0.005*"pap" + '
  '0.005*"not" + 0.005*"job" + 0.005*"need"')]


In [18]:
#Run and train LDA model on the document term matrix for 4 topics
ldamodel4 = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, passes = 100, random_state = 42)

In [19]:
#Print topic number and most contributing words
pprint(ldamodel4.print_topics(num_words=15))

[(0,
  '0.016*"school" + 0.013*"student" + 0.012*"stream" + 0.008*"good" + '
  '0.008*"go" + 0.007*"normal" + 0.007*"one" + 0.006*"uturn" + 0.006*"teacher" '
  '+ 0.005*"express" + 0.005*"lose" + 0.005*"minister" + 0.005*"system" + '
  '0.005*"mix" + 0.005*"like"'),
 (1,
  '0.014*"singapore" + 0.011*"like" + 0.009*"bad" + 0.007*"year" + '
  '0.007*"even" + 0.007*"singaporean" + 0.006*"civil" + 0.006*"education" + '
  '0.006*"system" + 0.006*"many" + 0.005*"chinese" + 0.005*"or" + '
  '0.005*"government" + 0.005*"world" + 0.005*"not"'),
 (2,
  '0.011*"student" + 0.011*"take" + 0.011*"good" + 0.009*"get" + '
  '0.007*"change" + 0.007*"policy" + 0.007*"streaming" + 0.007*"need" + '
  '0.007*"stream" + 0.006*"job" + 0.006*"education" + 0.006*"people" + '
  '0.006*"pap" + 0.005*"also" + 0.004*"want"'),
 (3,
  '0.015*"student" + 0.009*"system" + 0.009*"stream" + 0.008*"class" + '
  '0.007*"not" + 0.007*"school" + 0.007*"many" + 0.007*"normal" + '
  '0.006*"nonchinese" + 0.006*"would" + 0.006

<div class='alert alert-block alert-warning'>
    There does not seem to be distinct separation of topics using gensim. There is a lot of overlap for the top contributing words for each topic.
</div>

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [20]:
#Visualise topics and most relevant terms
pyLDAvis.gensim.prepare(ldamodel3, doc_term_matrix, dictionary)

C:\Users\kwanw\Anaconda3\envs\tf\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.117224  0.011334       1        1  51.423656
2      0.045777 -0.083302       2        1  26.979166
0      0.071448  0.071968       3        1  21.597172, topic_info=     Category       Freq         Term      Total  loglift  logprob
162   Default  48.000000       school  48.000000  30.0000  30.0000
253   Default  26.000000         take  26.000000  29.0000  29.0000
38    Default  34.000000          get  34.000000  28.0000  28.0000
657   Default  26.000000      chinese  26.000000  27.0000  27.0000
94    Default  18.000000       policy  18.000000  26.0000  26.0000
1443  Default  10.000000        uturn  10.000000  25.0000  25.0000
658   Default  19.000000        civil  19.000000  24.0000  24.0000
39    Default  50.000000         good  50.000000  23.0000  23.0000
278   Default  14.000000       parent  14.000000  22.0000  22.0000
70    Default  68.000000      student  68.000000  21.0000  21.0000
58    Default  12.000000      subject  12.000000  20.0000  20.0000
294   Default  28.000000          one  28.000000  19.0000  19.0000
100   Default  57.000000       stream  57.000000  18.0000  18.0000
935   Default   9.000000    different   9.000000  17.0000  17.0000
21    Default  57.000000    singapore  57.000000  16.0000  16.0000
45    Default  11.000000         lose  11.000000  15.0000  15.0000
1451  Default   6.000000          grc   6.000000  14.0000  14.0000
11    Default  27.000000           go  27.000000  13.0000  13.0000
274   Default  17.000000        child  17.000000  12.0000  12.0000
177   Default  20.000000         need  20.000000  11.0000  11.0000
31    Default  25.000000       change  25.000000  10.0000  10.0000
777   Default  16.000000  restructure  16.000000   9.0000   9.0000
179   Default  10.000000        right  10.000000   8.0000   8.0000
245   Default  33.000000       normal  33.000000   7.0000   7.0000
590   Default  10.000000     election  10.000000   6.0000   6.0000
29    Default   8.000000         base   8.000000   5.0000   5.0000
271   Default  14.000000      teacher  14.000000   4.0000   4.0000
373   Default   6.000000        money   6.000000   3.0000   3.0000
116   Default  22.000000          pap  22.000000   2.0000   2.0000
238   Default   7.000000            5   7.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
1794   Topic3   3.654568       reduce   5.171160   1.1855  -6.1370
45     Topic3   7.313029         lose  11.651715   1.0668  -5.4433
162    Topic3  22.212776       school  48.891506   0.7437  -4.3323
73     Topic3   5.179068      another   8.791605   1.0034  -5.7884
271    Topic3   7.222950      teacher  14.145204   0.8605  -5.4557
590    Topic3   5.881544     election  10.670496   0.9369  -5.6612
294    Topic3  11.489812          one  28.225368   0.6338  -4.9915
274    Topic3   7.959821        child  17.048271   0.7710  -5.3586
11     Topic3  10.769333           go  27.337296   0.6011  -5.0563
278    Topic3   6.975291       parent  14.661394   0.7898  -5.4906
100    Topic3  17.058298       stream  57.552261   0.3166  -4.5963
316    Topic3   3.063950           hv   4.334627   1.1857  -6.3133
521    Topic3   3.056996     standard   4.335292   1.1833  -6.3155
70     Topic3  17.648949      student  68.414856   0.1777  -4.5623
1000   Topic3   5.805010          low  11.992527   0.8070  -5.6743
245    Topic3  10.170355       normal  33.307114   0.3463  -5.1135
32     Topic3   8.680455        class  26.554245   0.4145  -5.2719
39     Topic3  11.682540         good  50.885422   0.0611  -4.9749
59     Topic3   9.138700       system  45.346012  -0.0692  -5.2205
140    Topic3   6.627727         make  23.390152   0.2716  -5.5417
35     Topic3   7.802346    education  41.760113  -0.1449  -5.3786
318    Topic3   7.234293         many  37.047676  -0.1008  -5.4541
297    Topic3   6.369054          say  24.726048   0.1762  -5.5815
87

## Include Bigrams

In [21]:
from gensim.models import Phrases

In [22]:
#Convert each comment into a list of unigrams
text = [line.split() for line in comments_df['comments']]

In [23]:
# Add bigrams to documents (minimum 10 occurances)
bigrams = Phrases(text, min_count=10)

for index in range(len(text)):
    #run through tokens including the bigrams 
    for word in bigrams[text[index]]:
        #Check if token is a bigram
        if '_' in word:
            #Add bigram to the doc
            text[index].append(word)

In [24]:
#Create a new column for the comments with bigrams, convert the lists of unigrams and bigrams into individual comments
comments_df['bigram_text'] = [' '.join(line) for line in text] 

In [25]:
def clean2(text):
    '''
    This function takes in a string and returns the string with stopwords and punctuation removed, plus words are lemmatized unless they are pronouns.
    '''
    #Remove stopwords and punctuation
    removed_stopwords = " ".join(i for i in text.lower().split() if i not in nltk_stops)
    removed_punctuation = "".join(i for i in removed_stopwords if i not in punctuation or i == '_')
    
    #Tokenize words
    doc = nlp(removed_punctuation)
    #Lemmatize words
    lemmatized = " ".join(token.lemma_ if token.pos_ != 'PRON' else token.text for token in doc)
    return lemmatized

In [26]:
#Add the cleaned bigram text to the DataFrame
comments_df['cleaned_bigram_text'] = comments_df['bigram_text'].apply(clean2)

In [27]:
#Convert each comment into a list of unigrams/bigrams
text_list_2 = [text.split() for text in comments_df['cleaned_bigram_text']]

In [28]:
#Create a gensim dictionary
dictionary_bigram = corpora.Dictionary(text_list_2)
print(dictionary_bigram)

Dictionary(2146 unique tokens: ['apply', 'army', 'back', 'china', 'citizenship']...)


<div class='alert alert-block alert-warning'>
    Only 11 more words than previously, seems unlikely to have much impact.
</div>

In [29]:
#Convert corpus into Document Term Matrix using dictionary prepared above.
dtm_2 = [dictionary_bigram.doc2bow(doc) for doc in text_list_2]

In [30]:
#Run and train LDA model on the document term matrix for 2 topics
lda_bg = Lda(dtm_2, num_topics=2, id2word = dictionary_bigram, passes = 100, random_state = 42)

In [31]:
#Print topic number and most contributing words
pprint(lda_bg.show_topics(num_words=15))

[(0,
  '0.016*"student" + 0.012*"stream" + 0.011*"good" + 0.009*"school" + '
  '0.007*"normal" + 0.006*"express" + 0.006*"one" + 0.006*"system" + '
  '0.006*"kid" + 0.006*"many" + 0.006*"take" + 0.006*"education" + 0.005*"get" '
  '+ 0.005*"streaming" + 0.005*"like"'),
 (1,
  '0.012*"singapore" + 0.011*"_" + 0.009*"like" + 0.007*"year" + 0.007*"bad" + '
  '0.007*"even" + 0.007*"chinese" + 0.006*"singaporean" + 0.006*"world" + '
  '0.005*"government" + 0.005*"system" + 0.005*"civil" + 0.005*"foreign" + '
  '0.005*"education" + 0.005*"do"')]


In [32]:
#Run and train LDA model on the document term matrix for 3 topics
lda_bg_3 = Lda(dtm_2, num_topics=3, id2word = dictionary_bigram, passes = 100, random_state = 42)

In [33]:
#Print topic number and most contributing words
pprint(lda_bg_3.show_topics(num_words=15))

[(0,
  '0.014*"stream" + 0.014*"student" + 0.012*"good" + 0.011*"normal" + '
  '0.009*"express" + 0.008*"school" + 0.007*"many" + 0.007*"kid" + 0.007*"one" '
  '+ 0.007*"go" + 0.006*"take" + 0.006*"like" + 0.005*"year" + 0.005*"people" '
  '+ 0.005*"study"'),
 (1,
  '0.007*"minister" + 0.007*"local" + 0.007*"foreign" + 0.006*"job" + '
  '0.006*"university" + 0.005*"issue" + 0.005*"singapore" + 0.005*"take" + '
  '0.005*"elite" + 0.005*"chinese" + 0.005*"change" + 0.005*"like" + '
  '0.004*"year" + 0.004*"do" + 0.004*"not"'),
 (2,
  '0.010*"system" + 0.010*"singapore" + 0.009*"_" + 0.009*"like" + '
  '0.008*"student" + 0.008*"school" + 0.007*"education" + 0.007*"bad" + '
  '0.006*"get" + 0.005*"people" + 0.005*"even" + 0.005*"stream" + '
  '0.005*"class" + 0.005*"not" + 0.005*"change"')]


In [34]:
#Run and train LDA model on the document term matrix for 4 topics
lda_bg_4 = Lda(dtm_2, num_topics=3, id2word = dictionary_bigram, passes = 100, random_state = 42)

In [35]:
#Print topic number and most contributing words
pprint(lda_bg_4.show_topics(num_words=15))

[(0,
  '0.014*"stream" + 0.014*"student" + 0.012*"good" + 0.011*"normal" + '
  '0.009*"express" + 0.008*"school" + 0.007*"many" + 0.007*"kid" + 0.007*"one" '
  '+ 0.007*"go" + 0.006*"take" + 0.006*"like" + 0.005*"year" + 0.005*"people" '
  '+ 0.005*"study"'),
 (1,
  '0.007*"minister" + 0.007*"local" + 0.007*"foreign" + 0.006*"job" + '
  '0.006*"university" + 0.005*"issue" + 0.005*"singapore" + 0.005*"take" + '
  '0.005*"elite" + 0.005*"chinese" + 0.005*"change" + 0.005*"like" + '
  '0.004*"year" + 0.004*"do" + 0.004*"not"'),
 (2,
  '0.010*"system" + 0.010*"singapore" + 0.009*"_" + 0.009*"like" + '
  '0.008*"student" + 0.008*"school" + 0.007*"education" + 0.007*"bad" + '
  '0.006*"get" + 0.005*"people" + 0.005*"even" + 0.005*"stream" + '
  '0.005*"class" + 0.005*"not" + 0.005*"change"')]


<div class='alert alert-block alert-warning'>
    There might be a bit of indication of politics for 2 and 3 topics, with words such as government, system, civil and minister, local, foreign respectively. However, there is still quite a bit of overlap and it feels like quite the stretch. I'll take a look a the top relevant terms and their relative contributions to get a better picture.
</div>

In [36]:
#Visualise topics and most relevant terms for 2 topics
pyLDAvis.gensim.prepare(lda_bg, dtm_2, dictionary_bigram)

PreparedData(topic_coordinates=             x    y  topics  cluster       Freq
topic                                          
0      0.10284  0.0       1        1  55.107574
1     -0.10284  0.0       2        1  44.892426, topic_info=     Category       Freq         Term      Total  loglift  logprob
71    Default  69.000000      student  69.000000  30.0000  30.0000
637   Default  40.000000            _  40.000000  29.0000  29.0000
662   Default  24.000000      chinese  24.000000  28.0000  28.0000
248   Default  33.000000       normal  33.000000  27.0000  27.0000
101   Default  58.000000       stream  58.000000  26.0000  26.0000
21    Default  54.000000    singapore  54.000000  25.0000  25.0000
245   Default  28.000000      express  28.000000  24.0000  24.0000
40    Default  52.000000         good  52.000000  23.0000  23.0000
837   Default  20.000000        world  20.000000  22.0000  22.0000
43    Default  26.000000          kid  26.000000  21.0000  21.0000
663   Default  18.000000        civil  18.000000  20.0000  20.0000
10    Default  18.000000      foreign  18.000000  19.0000  19.0000
785   Default  15.000000  restructure  15.000000  18.0000  18.0000
57    Default  24.000000    streaming  24.000000  17.0000  17.0000
797   Default  14.000000      service  14.000000  16.0000  16.0000
122   Default  27.000000  singaporean  27.000000  15.0000  15.0000
505   Default  17.000000           or  17.000000  14.0000  14.0000
685   Default  14.000000          due  14.000000  13.0000  13.0000
297   Default  29.000000          one  29.000000  12.0000  12.0000
546   Default  13.000000       reform  13.000000  11.0000  11.0000
76    Default  32.000000          bad  32.000000  10.0000  10.0000
277   Default  18.000000        child  18.000000   9.0000   9.0000
798   Default  12.000000          sex  12.000000   8.0000   8.0000
686   Default  13.000000       due_to  13.000000   7.0000   7.0000
371   Default  15.000000         call  15.000000   6.0000   6.0000
820   Default  11.000000         type  11.000000   5.0000   5.0000
684   Default  11.000000           dr  11.000000   4.0000   4.0000
3     Default  12.000000        china  12.000000   3.0000   3.0000
56    Default  16.000000        still  16.000000   2.0000   2.0000
22    Default  17.000000        study  17.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
3      Topic2  11.924816        china  12.559410   0.7491  -5.7088
774    Topic2   6.668499        power   7.065057   0.7431  -6.2900
701    Topic2   6.637897         face   7.065229   0.7385  -6.2946
1108   Topic2   5.884200    christian   6.280427   0.7357  -6.4151
409    Topic2   5.884047          law   6.280427   0.7357  -6.4152
772    Topic2   5.877566      politic   6.280451   0.7346  -6.4163
807    Topic2   5.877221        staff   6.280464   0.7345  -6.4163
505    Topic2  16.087862           or  17.272337   0.7299  -5.4094
150    Topic2   8.021301          act   8.635997   0.7271  -6.1053
371    Topic2  14.161609         call  15.706556   0.6974  -5.5369
536    Topic2  10.576156    unethical  11.780108   0.6931  -5.8288
21     Topic2  43.056602    singapore  54.246120   0.5699  -4.4249
122    Topic2  22.827579  singaporean  27.506126   0.6145  -5.0595
76     Topic2  24.980625          bad  32.238670   0.5458  -4.9693
479    Topic2  18.691477   government  24.377260   0.5353  -5.2594
258    Topic2  24.982216         year  35.407639   0.4521  -4.9693
174    Topic2  24.626923         even  36.995571   0.3939  -4.9836
270    Topic2  16.562851           do  22.021160   0.5161  -5.3803
88     Topic2  30.773922         like  54.365303   0.2318  -4.7608
832    Topic2  11.952796        whole  14.143449   0.6326  -5.7065
398    Topic2  14.864252        elite  23.622133   0.3377  -5.4885
171    Topic2  12.643136          big  17.305752   0.4870  -5.6503
60     Topic2  18.090097       system  44.981453  -0.1100  -5.2921
300    Topic2  14.297358          say  24.419880   0.2656  -5.5273
35     Topic2

In [37]:
#Visualise topics and most relevant terms for 3 topics
pyLDAvis.gensim.prepare(lda_bg_3, dtm_2, dictionary_bigram)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.110062  0.040530       1        1  49.151577
0     -0.014253 -0.102309       2        1  33.808716
1     -0.095810  0.061779       3        1  17.039711, topic_info=     Category       Freq        Term      Total  loglift  logprob
248   Default  33.000000      normal  33.000000  30.0000  30.0000
101   Default  59.000000      stream  59.000000  29.0000  29.0000
245   Default  28.000000     express  28.000000  28.0000  28.0000
627   Default  14.000000       local  14.000000  27.0000  27.0000
10    Default  16.000000     foreign  16.000000  26.0000  26.0000
256   Default  26.000000        take  26.000000  25.0000  25.0000
40    Default  53.000000        good  53.000000  24.0000  24.0000
198   Default  24.000000    minister  24.000000  23.0000  23.0000
398   Default  23.000000       elite  23.000000  22.0000  22.0000
86    Default  12.000000       issue  12.000000  21.0000  21.0000
43    Default  26.000000         kid  26.000000  20.0000  20.0000
25    Default  18.000000  university  18.000000  19.0000  19.0000
71    Default  71.000000     student  71.000000  18.0000  18.0000
662   Default  24.000000     chinese  24.000000  17.0000  17.0000
87    Default  27.000000         job  27.000000  16.0000  16.0000
31    Default  26.000000      change  26.000000  15.0000  15.0000
496   Default  11.000000        life  11.000000  14.0000  14.0000
22    Default  16.000000       study  16.000000  13.0000  13.0000
59    Default  13.000000     subject  13.000000  12.0000  12.0000
637   Default  43.000000           _  43.000000  11.0000  11.0000
1108  Default   5.000000   christian   5.000000  10.0000  10.0000
621   Default  11.000000      degree  11.000000   9.0000   9.0000
337   Default   9.000000           u   9.000000   8.0000   8.0000
297   Default  29.000000         one  29.000000   7.0000   7.0000
11    Default  28.000000          go  28.000000   6.0000   6.0000
430   Default   8.000000     without   8.000000   5.0000   5.0000
424   Default  12.000000      salary  12.000000   4.0000   4.0000
12    Default   8.000000    graduate   8.000000   3.0000   3.0000
1117  Default   4.000000     harvard   4.000000   2.0000   2.0000
1133  Default   4.000000      oxford   4.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
1140   Topic3   2.887551     realise   4.209554   1.3927  -6.1583
10     Topic3   9.394351     foreign  16.563789   1.2025  -4.9786
430    Topic3   4.902340     without   8.621496   1.2051  -5.6290
198    Topic3  10.083263    minister  24.703804   0.8735  -4.9078
25     Topic3   7.566761  university  18.440849   0.8788  -5.1949
247    Topic3   2.900214      gifted   4.256464   1.3860  -6.1539
1131   Topic3   2.866747       never   4.214305   1.3843  -6.1655
87     Topic3   8.666508         job  27.412319   0.6181  -5.0592
694    Topic3   4.876793  especially  11.203957   0.9378  -5.6342
1370   Topic3   2.838857       raise   4.273366   1.3606  -6.1753
12     Topic3   4.221338    graduate   8.698821   1.0466  -5.7785
587    Topic3   3.550438         ong   6.609361   1.1482  -5.9516
398    Topic3   6.896191       elite  23.397072   0.5480  -5.2877
256    Topic3   7.158856        take  26.437485   0.4632  -5.2503
662    Topic3   6.549529     chinese  24.279436   0.4594  -5.3393
774    Topic3   3.540966       power   6.611917   1.1451  -5.9543
31     Topic3   6.297949      change  26.011379   0.3513  -5.3785
428    Topic3   4.187777        talk  10.449335   0.8553  -5.7865
270    Topic3   5.546704          do  21.731995   0.4040  -5.5055
21     Topic3   7.290053   singapore  55.513508  -0.2605  -5.2322
621    Topic3   4.198335      degree  11.075020   0.7996  -5.7840
258    Topic3   5.999699        year  35.334854  -0.0035  -5.4270
307    Topic3   4.855530        also  19.627401   0.3728  -5.6386
201    Topic3   4.469574         new  15.408352   0.5320 

In [38]:
#Retrieve topics and probabilities for each, for the first comment
lda_bg_3.get_document_topics(dictionary_bigram.doc2bow(text_list_2[0]))

[(0, 0.011714848), (1, 0.97677267), (2, 0.011512484)]

In [39]:
#Create function to sort by the second item in the tuple
def sort_by_second(tuple_item):
    return tuple_item[1]

#Retrieve the most probable topic, for the first comment
sorted(lda_bg_3.get_document_topics(dictionary_bigram.doc2bow(text_list_2[0])),key=sort_by_second, reverse= True)[0][0]

1

In [40]:
#Create function to retrieve most probable topic
def get_topic(text_entry):
    return sorted(lda_bg_3.get_document_topics(dictionary_bigram.doc2bow(text_entry)),key=sort_by_second, reverse= True)[0][0]

In [41]:
#Add the topics to the DataFrame
comments_df['topics'] = [get_topic(comment) for comment in text_list_2]

In [42]:
#Print the number of comments for each topic
for i in range(3):
    print('No. of comments for topic {}: '.format(i),len(comments_df[comments_df['topics']==i]))

No. of comments for topic 0:  96
No. of comments for topic 1:  58
No. of comments for topic 2:  111


In [43]:
import random

In [44]:
def show_random_5(topic_num):
    '''
    This function takes in a topic number and prints out 5 random comments belonging to that topic
    '''
    #for reproducibility
    random.seed(99)
    #randomly pick 5 indexes
    for index in random.sample(range(len(comments_df[comments_df['topics']==topic_num])),5):
        #Print comment with the selected index
        print(comments_df[comments_df['topics']==topic_num].iloc[index]['comments'])
        print('-----'*20)
        print('\n')

In [45]:
#Show 5 random comments from topic 0
show_random_5(0)

No one takes Yahoo polls seriously. Are you new here?
----------------------------------------------------------------------------------------------------


@ Peter, most of ur tribe members cannot make it, so how to break bond?
----------------------------------------------------------------------------------------------------


No more streaming into classes but segregating students into subjects means a different way to stream!
----------------------------------------------------------------------------------------------------


China action good good
----------------------------------------------------------------------------------------------------


Look at how many students suffer mental problems. All thanks to the stressful education system
----------------------------------------------------------------------------------------------------




In [46]:
#Show 5 random comments from topic 1
show_random_5(1)

Why need to study much, at the end, the 4ligners getting the jobs
----------------------------------------------------------------------------------------------------


WHAT ABOUT SEPS SCHOOLS?
this is THE major stigma that leads to elitism that forms the distinct gaps!
such move will only distinguish this elite groups further away from the mass, holding them 'untouchable' in moral and social.
likewise, what about educators themselves????
More
----------------------------------------------------------------------------------------------------


CEOs tell Trump they are hiring more Americans without college degrees:
WASHINGTON (Reuters) - Chief executives of major companies said at a White House forum on Wednesday that they are hiring more Americans without college degrees as they search to find increasingly scarce applicants for open jobs.
----------------------------------------------------------------------------------------------------


Only 1diots like you take Yahoo polls serious

In [47]:
#Show 5 random comments from topic 2
show_random_5(2)

After election, things might change again...
----------------------------------------------------------------------------------------------------


Sounds like the GRC system!
----------------------------------------------------------------------------------------------------


Even the teachers
----------------------------------------------------------------------------------------------------


Not a problem, as long as the schools are top performing and have a challenging curriculum. I hope Singapore doesn’t marginalize their school system
----------------------------------------------------------------------------------------------------


To Everyone in this Website, Especially PAP, Opposition Parties & All Singaporean,

To improve our competitiveness in Global Economy , We really must REVAMP our entire school education system , in actual fact, it should have been Done it in over 20 years ago, during the 1990s .

From this website on “ Subject-Based Banding to replace streaming in

<div class='alert alert-block alert-warning'>
    There is still a lot of overlap and topic distribution isn't clear. Perhaps unigrams without lemmatization (as was done previously for sci-kit learn's LDA and NMF topic modelling) would provide better results.
</div>

## Minimal preprocessing (no lemmatization)

In [48]:
def remove_stopwords_punctuation(article):
    '''
    This function takes in a string and returns the string with stopwords and punctuation removed
    '''
    removed_stopwords = " ".join(word for word in article.lower().split() if word not in nltk_stops)
    removed_punctuation = ''.join(letter for letter in removed_stopwords if letter not in punctuation)
    return removed_punctuation

In [49]:
#Remove stopwaords and punctuation
text_3 = [remove_stopwords_punctuation(article) for article in comments_df['comments']]

In [50]:
#Convert each comment into a list of unigrams
text_list_3 = [comment.split() for comment in text_3]

In [51]:
#Crate gensim dictionary
dictionary_3 = corpora.Dictionary(text_list_3)
print(dictionary_3)

Dictionary(2575 unique tokens: ['apply', 'army', 'back', 'china', 'citizenship']...)


In [52]:
#Convert corpus into Document Term Matrix using dictionary prepared above.
dtm_3 = [dictionary_3.doc2bow(comment) for comment in text_list_3]

In [53]:
#Run and train LDA model on the document term matrix for 2 topics
lda_min = Lda(dtm_3,num_topics=2,id2word=dictionary_3, passes=100,random_state=42)

In [54]:
#Print topic number and most contributing words
pprint(lda_min.show_topics())

[(0,
  '0.009*"students" + 0.008*"like" + 0.007*"singapore" + 0.007*"streaming" + '
  '0.007*"even" + 0.006*"good" + 0.006*"education" + 0.006*"system" + '
  '0.006*"stream" + 0.005*"many"'),
 (1,
  '0.005*"singapore" + 0.005*"streaming" + 0.005*"students" + 0.004*"system" + '
  '0.004*"foreign" + 0.004*"education" + 0.004*"years" + 0.003*"local" + '
  '0.003*"pap" + 0.003*"one"')]


In [55]:
#Run and train LDA model on the document term matrix for 3 topics
lda_min_3 = Lda(dtm_3,num_topics=3,id2word=dictionary_3, passes=100,random_state=42)

In [56]:
#Print topic number and most contributing words
pprint(lda_min_3.show_topics())

[(0,
  '0.009*"singapore" + 0.008*"like" + 0.008*"students" + 0.007*"good" + '
  '0.007*"streaming" + 0.007*"even" + 0.007*"system" + 0.006*"education" + '
  '0.005*"many" + 0.005*"people"'),
 (1,
  '0.005*"streaming" + 0.005*"education" + 0.005*"pap" + 0.004*"social" + '
  '0.004*"polls" + 0.004*"uturn" + 0.004*"years" + 0.004*"one" + '
  '0.003*"always" + 0.003*"system"'),
 (2,
  '0.012*"normal" + 0.010*"students" + 0.010*"stream" + 0.009*"kids" + '
  '0.009*"express" + 0.006*"streaming" + 0.006*"like" + 0.005*"foreign" + '
  '0.005*"many" + 0.005*"singapore"')]


In [57]:
#Run and train LDA model on the document term matrix for 4 topics
lda_min_4 = Lda(dtm_3,num_topics=4,id2word=dictionary_3, passes=100,random_state=42)

In [58]:
#Print topic number and most contributing words
pprint(lda_min_4.show_topics())

[(0,
  '0.010*"singapore" + 0.009*"like" + 0.007*"students" + 0.007*"even" + '
  '0.006*"people" + 0.006*"system" + 0.006*"good" + 0.006*"streaming" + '
  '0.005*"singaporean" + 0.005*"many"'),
 (1,
  '0.006*"pap" + 0.004*"make" + 0.004*"education" + 0.004*"low" + 0.004*"take" '
  '+ 0.003*"better" + 0.003*"like" + 0.003*"streaming" + 0.003*"days" + '
  '0.003*"question"'),
 (2,
  '0.020*"normal" + 0.016*"express" + 0.014*"kids" + 0.014*"stream" + '
  '0.010*"students" + 0.008*"many" + 0.007*"streaming" + 0.006*"like" + '
  '0.005*"school" + 0.004*"look"'),
 (3,
  '0.007*"education" + 0.007*"streaming" + 0.007*"students" + '
  '0.007*"singapore" + 0.006*"system" + 0.006*"good" + 0.005*"schools" + '
  '0.005*"foreign" + 0.005*"like" + 0.004*"years"')]


In [59]:
pyLDAvis.gensim.prepare(lda_min, dtm_3, dictionary_3)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.079544  0.0       1        1  67.552361
1     -0.079544  0.0       2        1  32.447643, topic_info=     Category       Freq          Term      Total  loglift  logprob
700   Default  11.000000         local  11.000000  30.0000  30.0000
293   Default  23.000000           bad  23.000000  29.0000  29.0000
122   Default   6.000000        policy   6.000000  28.0000  28.0000
1784  Default   6.000000         polls   6.000000  27.0000  27.0000
465   Default  23.000000   singaporean  23.000000  26.0000  26.0000
10    Default  16.000000       foreign  16.000000  25.0000  25.0000
2347  Default   6.000000         uturn   6.000000  24.0000  24.0000
629   Default  31.000000        stream  31.000000  23.0000  23.0000
1665  Default   6.000000            ng   6.000000  22.0000  22.0000
185   Default  20.000000          need  20.000000  21.0000  21.0000
370   Default   7.000000           lol   7.000000  20.0000  20.0000
1173  Default   9.000000  universities   9.000000  19.0000  19.0000
556   Default  17.000000            or  17.000000  18.0000  18.0000
1250  Default   4.000000     cambridge   4.000000  17.0000  17.0000
1266  Default   4.000000       harvard   4.000000  16.0000  16.0000
1284  Default   4.000000        oxford   4.000000  15.0000  15.0000
2495  Default   4.000000          liao   4.000000  14.0000  14.0000
1744  Default   4.000000          here   4.000000  13.0000  13.0000
77    Default   9.000000        cannot   9.000000  12.0000  12.0000
278   Default  14.000000          make  14.000000  11.0000  11.0000
292   Default   5.000000         again   5.000000  10.0000  10.0000
875   Default  15.000000   restructure  15.000000   9.0000   9.0000
408   Default  10.000000           see  10.000000   8.0000   8.0000
896   Default  16.000000        social  16.000000   7.0000   7.0000
1032  Default   4.000000    parliament   4.000000   6.0000   6.0000
236   Default   4.000000             7   4.000000   5.0000   5.0000
1695  Default   4.000000            ge   4.000000   4.0000   4.0000
2014  Default   4.000000          rich   4.000000   3.0000   3.0000
211   Default  13.000000           new  13.000000   2.0000   2.0000
2     Default   7.000000          back   7.000000   1.0000   1.0000
...       ...        ...           ...        ...      ...      ...
113    Topic2   3.829802            vs   4.806707   0.8983  -6.4877
77     Topic2   6.182381        cannot   9.926171   0.6521  -6.0088
408    Topic2   6.234370           see  10.717496   0.5837  -6.0005
278    Topic2   7.443518          make  14.473763   0.4605  -5.8232
2      Topic2   4.983218          back   7.772399   0.6810  -6.2245
126    Topic2   4.362182  singaporeans   6.298905   0.7581  -6.3576
896    Topic2   7.614708        social  16.042229   0.3804  -5.8005
211    Topic2   6.880470           new  13.790316   0.4303  -5.9019
97     Topic2   5.824079      policies  10.804104   0.5076  -6.0686
19     Topic2  13.506938     singapore  50.901997  -0.2012  -5.2273
121    Topic2   8.458198           pap  22.282585   0.1569  -5.6954
317    Topic2   8.225579           one  21.529383   0.1634  -5.7233
55     Topic2  11.834250     streaming  48.045864  -0.2756  -5.3596
87     Topic2   5.089879         issue   8.552175   0.6066  -6.2033
59     Topic2  10.253859        system  41.158764  -0.2642  -5.5029
144    Topic2   6.155649            it  13.141006   0.3672  -6.0132
72     Topic2  11.518245      students  56.135593  -0.4583  -5.3866
931    Topic2   8.914268         years  31.011633  -0.1212  -5.6429
35     Topic2   9.462043     education  40.523621  -0.3291  -5.5833
738    Topic2   7.554717       chinese  23.275604   0.0003  -5.8084
327    Topic2   6.918266          also  18.596149   0.1368  -5.8964
31     Topic2   6.671895        change  17.845833   0.1417  -5.9326
3      Topic2   5.644450         china  11.644317   0.4014  -6.0999
15     Topic2   7.676

In [60]:
pyLDAvis.gensim.prepare(lda_min_3, dtm_3, dictionary_3)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.095821  0.048864       1        1  58.210087
2     -0.000799 -0.098971       2        1  23.250160
1     -0.095022  0.050107       3        1  18.539755, topic_info=     Category       Freq            Term      Total  loglift  logprob
261   Default  29.000000          normal  29.000000  30.0000  30.0000
629   Default  28.000000          stream  28.000000  29.0000  29.0000
42    Default  22.000000            kids  22.000000  28.0000  28.0000
257   Default  25.000000         express  25.000000  27.0000  27.0000
10    Default  16.000000         foreign  16.000000  26.0000  26.0000
72    Default  55.000000        students  55.000000  25.0000  25.0000
465   Default  24.000000     singaporean  24.000000  24.0000  24.0000
121   Default  22.000000             pap  22.000000  23.0000  23.0000
2347  Default   5.000000           uturn   5.000000  22.0000  22.0000
1784  Default   6.000000           polls   6.000000  21.0000  21.0000
253   Default   6.000000               5   6.000000  20.0000  20.0000
738   Default  24.000000         chinese  24.000000  19.0000  19.0000
896   Default  16.000000          social  16.000000  18.0000  18.0000
700   Default  12.000000           local  12.000000  17.0000  17.0000
659   Default   8.000000        election   8.000000  16.0000  16.0000
2495  Default   4.000000            liao   4.000000  15.0000  15.0000
370   Default   7.000000             lol   7.000000  14.0000  14.0000
1500  Default   7.000000  stratification   7.000000  13.0000  13.0000
556   Default  18.000000              or  18.000000  12.0000  12.0000
122   Default   7.000000          policy   7.000000  11.0000  11.0000
221   Default  12.000000          always  12.000000  10.0000  10.0000
60    Default  17.000000             top  17.000000   9.0000   9.0000
144   Default  12.000000              it  12.000000   8.0000   8.0000
1665  Default   6.000000              ng   6.000000   7.0000   7.0000
1266  Default   4.000000         harvard   4.000000   6.0000   6.0000
1284  Default   4.000000          oxford   4.000000   5.0000   5.0000
1250  Default   4.000000       cambridge   4.000000   4.0000   4.0000
77    Default   9.000000          cannot   9.000000   3.0000   3.0000
87    Default   8.000000           issue   8.000000   2.0000   2.0000
875   Default  16.000000     restructure  16.000000   1.0000   1.0000
...       ...        ...             ...        ...      ...      ...
137    Topic3   2.729126         changes   3.912713   1.3250  -6.2669
1032   Topic3   2.716391      parliament   3.916772   1.3193  -6.2715
663    Topic3   2.769348            mind   4.057500   1.3033  -6.2522
106    Topic3   3.357670         whether   5.538624   1.1848  -6.0596
122    Topic3   3.936417          policy   7.016563   1.1073  -5.9006
113    Topic3   2.743176              vs   4.590552   1.1704  -6.2617
370    Topic3   3.786776             lol   7.224708   1.0393  -5.9393
1500   Topic3   3.984540  stratification   7.844532   1.0079  -5.8884
87     Topic3   4.026443           issue   8.199373   0.9741  -5.8780
121    Topic3   7.130206             pap  22.523016   0.5351  -5.3065
896    Topic3   5.901629          social  16.115223   0.6807  -5.4956
221    Topic3   5.017025          always  12.184097   0.7980  -5.6580
84     Topic3   3.296782            find   6.223020   1.0499  -6.0779
1665   Topic3   3.369240              ng   6.581850   1.0156  -6.0562
167    Topic3   3.417586           place   6.881610   0.9853  -6.0419
97     Topic3   4.222258        policies  10.534978   0.7709  -5.8305
35     Topic3   7.135072       education  40.718582  -0.0564  -5.3058
55     Topic3   7.325284       streaming  47.869946  -0.1919  -5.2795
317    Topic3   5.145790             one  21.719700   0.2452  -5.6327
931    Topic3   5.244539           years  31.766319  -0.1160  -5.6137
59     Topic3   4.958959          system  41.83

<div class='alert alert-block alert-warning'>
    Even without lemmatization, gensim's LDA did not seem to be able to pull out distinct topics from the comments. It could be due to many the pool of comments being too small, and there being many similar terms across most of the comments.
</div>